In [1]:
import pandas as pd
import numpy as np
import os
import utils
from datetime import datetime
import importlib

In [2]:
buildings = ['am', 'f']

In [3]:
df = pd.read_parquet(f'data/hka-aqm-{buildings[1]}-combined-RAW.parquet') #switch between buildings for training
# drop channel_rssi and channel_index
df = df.drop(columns=['channel_rssi', 'channel_index'])

In [6]:
importlib.reload(utils)
performance_df = pd.read_csv('model_performances.csv')
d_model=256
nhead=8
num_layers=5 
dropout_pe=0.15
dropout_encoder=0.15
batch_size=1024
learning_rate=0.00031
epochs=25
y_feature = 'CO2'
aggregation_level = 'quarter_hour'
freq = 15 if aggregation_level == 'quarter_hour' else 30 if aggregation_level == 'half_hour' else 60
window_size = 5 * 60 // freq
for aggregation_level in ['quarter_hour', 'half_hour', 'hour']:
    for y_feature in ['CO2', 'VOC', 'hum', 'tmp', 'vis']:
        
        model, scaler, rmse, mae, me, mape, train_loss, val_loss, n_features = utils.create_multivariate_transformer_model_for_feature(df, d_model=d_model, nhead=nhead, num_layers=num_layers, dropout_pe=dropout_pe, dropout_encoder=dropout_encoder, batch_size=batch_size, learning_rate=learning_rate, epochs=epochs, y_feature=y_feature, aggregation_level=aggregation_level, window_size=window_size, selected_building='f')
        performance_df = performance_df.append({'model_name': 'multivariate_transformer','aggregation_level': aggregation_level, 'y_feature': y_feature, 'n_features': n_features, 'rmse': rmse, 'mae': mae, 'me': me, 'mape': mape, 'd_model': d_model, 'nhead': nhead, 'num_layers': num_layers, 'dropout_pe': dropout_pe,'dropout_encoder': dropout_encoder, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs, 'train_loss': train_loss, 'val_loss': val_loss, 'note': '2 decoder Linear layers with LeakyReLu, shuffled training data'}, ignore_index=True)
        performance_df.to_csv('model_performances.csv', index=False)
        

c:\Users\alexe\Desktop\IoT_Project\utils.py:1293: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_cpy['date_time_rounded'] = df_cpy['date_time'].dt.round('15T')
c:\Users\alexe\Desktop\IoT_Project\utils.py:1336: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(start=min_date, end=max_date, freq=freq)
c:\Users\alexe\Desktop\IoT_Project\utils.py:1321: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  device_df['device_id'] = device_df['device_id'].fillna(method='ffill')
c:\Users\alexe\Desktop\IoT_Project\utils.py:1324: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  device_df[[col for col in device_df.columns if col != 'date_time_rounded']] = device

extended data shape from (553093, 26) to (564981, 26)


c:\Users\alexe\Desktop\IoT_Project\utils.py:1352: FutureWarning: 'T' is deprecated and will be removed in a future version. Please use 'min' instead of 'T'.
  df_cpy['consecutive_data_point'] = (df_cpy['date_time_rounded'] - df_cpy['date_time_rounded'].shift(1)).dt.total_seconds() == pd.to_timedelta(freq).total_seconds()


training data cutoff:  2023-06-26 22:00:00
date_time_rounded    datetime64[ns]
device_id                    object
tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
snr                         float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday_sin                 float64
weekday_cos                 float64
month_sin                   float64
month_cos                   float64
time_sin                    float64
time_cos                    float64
semester_SS23               float64
semester_WS22/23            float64
semester_WS23/24            float64
group                

In [ ]:
# Reload the utils module if you have made changes
importlib.reload(utils)

# Read existing model performance records
performance_df = pd.read_csv('model_performances.csv')

# Define LSTM-specific hyperparameters
hidden_dim = 256
num_layers = 5
dropout = 0.3
batch_size = 4096
learning_rate = 0.00031
epochs = 25
aggregation_level = 'quarter_hour'
y_feature = 'CO2'
freq = 15 if aggregation_level == 'quarter_hour' else 30 if aggregation_level == 'half_hour' else 60
window_size = 5 * 60 // freq

# Loop over each feature to create and evaluate the LSTM model
for aggregation_level in ['quarter_hour', 'half_hour', 'hour']:
    for y_feature in ['CO2', 'VOC', 'hum', 'tmp', 'vis']:
        model, scaler, rmse, mae, me, mape, train_loss, val_loss, n_features = utils.create_multivariate_lstm_model_for_feature(
            df, 
            hidden_dim=hidden_dim, 
            num_layers=num_layers, 
            dropout=dropout, 
            batch_size=batch_size, 
            learning_rate=learning_rate, 
            epochs=epochs, 
            y_feature=y_feature, 
            aggregation_level=aggregation_level, 
            window_size=window_size
        )
        performance_df = performance_df.append({
            'model_name': 'multivariate_lstm',
            'aggregation_level': aggregation_level,
            'y_feature': y_feature,
            'n_features': n_features,
            'rmse': rmse,
            'mae': mae,
            'me': me,
            'mape': mape,
            'hidden_dim': hidden_dim,
            'num_layers': num_layers,
            'dropout': dropout,
            'batch_size': batch_size,
            'learning_rate': learning_rate,
            'epochs': epochs,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'note': 'LSTM model with device_id embedding'
        }, ignore_index=True)
        performance_df.to_csv('model_performances.csv', index=False)

extended data shape from (395288, 26) to (401657, 26)
training data cutoff:  2023-07-14 02:15:00
date_time_rounded    datetime64[ns]
device_id                    object
tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
snr                         float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday_sin                 float64
weekday_cos                 float64
month_sin                   float64
month_cos                   float64
time_sin                    float64
time_cos                    float64
semester_SS23               float64
semester_WS22/23            float64
sem

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


          actual   predicted
0     475.000000  475.310204
1     415.000002  415.023051
2     473.000000  477.090989
3     512.000001  529.613180
4     554.000002  574.161563
...          ...         ...
4091  420.000001  414.515299
4092  429.999999  430.388544
4093  425.000000  416.999382
4094  415.000002  413.225235
4095  412.999998  416.412842

[4096 rows x 2 columns]
Epoch 1/25, Training Loss: 0.3546
Epoch 1/25, Validation Loss: 0.0907
          actual   predicted
0     475.000000  474.009144
1     415.000002  418.210591
2     473.000000  470.695207
3     512.000001  521.129846
4     554.000002  574.822406
...          ...         ...
4091  420.000001  417.722072
4092  429.999999  431.246147
4093  425.000000  420.243562
4094  415.000002  413.374141
4095  412.999998  416.235154

[4096 rows x 2 columns]
Epoch 2/25, Training Loss: 0.0706
Epoch 2/25, Validation Loss: 0.0477
          actual   predicted
0     475.000000  473.969152
1     415.000002  423.640817
2     473.000000  470.90543

C:\Users\t-ehm\AppData\Local\Temp\ipykernel_15692\1996744831.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_df = performance_df.append({


extended data shape from (395288, 26) to (401657, 26)
training data cutoff:  2023-07-14 02:15:00
date_time_rounded    datetime64[ns]
device_id                    object
tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
snr                         float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday_sin                 float64
weekday_cos                 float64
month_sin                   float64
month_cos                   float64
time_sin                    float64
time_cos                    float64
semester_SS23               float64
semester_WS22/23            float64
sem

...

C:\Users\t-ehm\AppData\Local\Temp\ipykernel_15692\1996744831.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_df = performance_df.append({


extended data shape from (100645, 26) to (100793, 26)
training data cutoff:  2023-07-13 23:00:00
date_time_rounded    datetime64[ns]
device_id                    object
tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
snr                         float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday_sin                 float64
weekday_cos                 float64
month_sin                   float64
month_cos                   float64
time_sin                    float64
time_cos                    float64
semester_SS23               float64
semester_WS22/23            float64
sem

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


          actual   predicted
0       7.500007  206.393850
1     203.461538  192.169627
2       4.999994  456.866401
3      11.249995   45.806929
4      24.499993  116.804401
...          ...         ...
4091  266.384616  214.550772
4092  910.500029  522.002813
4093    8.499999  340.977483
4094    4.000001  145.782121
4095  379.043483  297.319932

[4096 rows x 2 columns]
Epoch 1/25, Training Loss: 0.9308
Epoch 1/25, Validation Loss: 0.7272
          actual   predicted
0       7.500007  195.064551
1     203.461538  286.057588
2       4.999994  363.406123
3      11.249995   35.270104
4      24.499993  139.370906
...          ...         ...
4091  266.384616  229.226148
4092  910.500029  976.970197
4093    8.499999   93.114324
4094    4.000001   -4.840214
4095  379.043483  152.999933

[4096 rows x 2 columns]
Epoch 2/25, Training Loss: 0.8885
Epoch 2/25, Validation Loss: 0.6999
          actual   predicted
0       7.500007  124.795465
1     203.461538  411.274653
2       4.999994  479.34410

C:\Users\t-ehm\AppData\Local\Temp\ipykernel_15692\1996744831.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_df = performance_df.append({


In [ ]:
!git add -A
!git commit -m "Updating models with 25 features instead of 26. Adding back snr feature"
!git push

[main abb8b35] Updating models with 25 features instead of 26. Adding back snr feature
 76 files changed, 958052 insertions(+), 949959 deletions(-)
 delete mode 100644 data/half_hour_23f_20ws_dataframe_v1.parquet
 create mode 100644 data/half_hour_25f_20ws_dataframe_v1.parquet
 delete mode 100644 data/half_hour_26f_20ws_dataframe_v1.parquet
 delete mode 100644 data/hour_23f_20ws_dataframe_v1.parquet
 create mode 100644 data/hour_25f_20ws_dataframe_v1.parquet
 delete mode 100644 data/hour_26f_20ws_dataframe_v1.parquet
 create mode 100644 data/lstm_multivariate_quarter_hour_25f_20ws_CO2_predictions.csv
 create mode 100644 data/lstm_multivariate_quarter_hour_25f_20ws_VOC_predictions.csv
 create mode 100644 data/lstm_multivariate_quarter_hour_25f_20ws_hum_predictions.csv
 create mode 100644 data/lstm_multivariate_quarter_hour_25f_20ws_tmp_predictions.csv
 create mode 100644 data/lstm_multivariate_quarter_hour_25f_20ws_vis_predictions.csv
 delete mode 100644 data/lstm_multivariate_quarter_h

error: RPC failed; curl 16 Error in the HTTP2 framing layer
send-pack: unexpected disconnect while reading sideband packet
fatal: the remote end hung up unexpectedly
Everything up-to-date


In [ ]:
import torch

device_ids = torch.tensor([0])

device = utils.get_device()

model = utils.load_transformer_model(y_feature="CO2", model_name = 'transformer_multivariate_quarter_hour_26f', device=device)
scaler = utils.load_scaler(y_feature="CO2", model_name = 'transformer_multivariate_quarter_hour_26f')

real_data = utils.load_dataframe(model_name='transformer_multivariate_quarter_hour_26f')
pd.set_option('display.max_columns', None)
real_data.iloc[7:27]

In [ ]:
data_df = real_data.iloc[7:27].drop(columns=['device_id', 'date_time_rounded'])

data_df_scaled = scaler.transform(data_df)

input_data = torch.tensor(data_df_scaled, dtype=torch.float32).view(-1, 20, data_df_scaled.shape[1])
print(input_data.shape)
device_ids = torch.tensor([0])

model.eval()
with torch.no_grad():
    output = model(input_data.to(device), device_ids.to(device))
    print(output)

    prediction = output.cpu().numpy().reshape(-1, 1)
    print(prediction)
    print(prediction.shape)
    zeroes_for_scaler = np.zeros((prediction.shape[0], 25))

    zeroes_for_scaler[:, 2] = prediction  # Insert predicted values into the correct column
    print(zeroes_for_scaler)
    inverse_transformed = scaler.inverse_transform(zeroes_for_scaler)
    predicted_unscaled = inverse_transformed[:, 2].round(2)
    print(predicted_unscaled)
    